In [15]:
import pandas as pd

In [16]:
import sqlite3

In [17]:
emp = pd.read_csv('C:\\Users\\Kumar\\Desktop\\New folder (2)\\Data-Science\\Python\\EDA\\employees.csv')


In [18]:
emp[emp['emp_no']==10010]

,emp_no,birth_date,first_name,last_name,gender,hire_date
9,10010,1963-06-01,Duangkaew,Piveteau,F,1989-08-24


In [19]:
emp_no_list = [10010,10011]
emp[emp['emp_no'].isin(emp_no_list)]

,emp_no,birth_date,first_name,last_name,gender,hire_date
9,10010,1963-06-01,Duangkaew,Piveteau,F,1989-08-24
10,10011,1953-11-07,Mary,Sluis,F,1990-01-22


In [20]:

conn = sqlite3.connect(':memory:')
emp.to_sql('employees', conn, index=False, if_exists='replace') 


1000

In [21]:
emp.shape

(1000, 6)

In [22]:
emp[emp['emp_no'].isin(emp_no_list)]

,emp_no,birth_date,first_name,last_name,gender,hire_date
9,10010,1963-06-01,Duangkaew,Piveteau,F,1989-08-24
10,10011,1953-11-07,Mary,Sluis,F,1990-01-22


In [23]:
salaries = pd.read_csv('C:\\Users\\Kumar\\Desktop\\New folder (2)\\Data-Science\\Python\\EDA\\Salaries.csv')


In [24]:
salaries.shape

(1000, 4)

In [25]:

salaries[salaries['emp_no'].isin(emp_no_list)]

,emp_no,salary,from_date,to_date
106,10010,72488,1996-11-24,1997-11-24
107,10010,74347,1997-11-24,1998-11-24
108,10010,75405,1998-11-24,1999-11-24
109,10010,78194,1999-11-24,2000-11-23
110,10010,79580,2000-11-23,2001-11-23
111,10010,80324,2001-11-23,9999-01-01
112,10011,42365,1990-01-22,1991-01-22
113,10011,44200,1991-01-22,1992-01-22
114,10011,48214,1992-01-22,1993-01-21
115,10011,50927,1993-01-21,1994-01-21


In [26]:

salaries.merge(emp,on='emp_no')



,emp_no,salary,from_date,to_date,birth_date,first_name,last_name,gender,hire_date
0,10001,60117,1986-06-26,1987-06-26,1953-09-02,Georgi,Facello,M,1986-06-26
1,10001,62102,1987-06-26,1988-06-25,1953-09-02,Georgi,Facello,M,1986-06-26
2,10001,66074,1988-06-25,1989-06-25,1953-09-02,Georgi,Facello,M,1986-06-26
3,10001,66596,1989-06-25,1990-06-25,1953-09-02,Georgi,Facello,M,1986-06-26
4,10001,66961,1990-06-25,1991-06-25,1953-09-02,Georgi,Facello,M,1986-06-26
...,...,...,...,...,...,...,...,...,...
995,10100,70464,1998-09-18,1999-09-18,1953-04-21,Hironobu,Haraldson,F,1987-09-21
996,10100,72343,1999-09-18,2000-09-17,1953-04-21,Hironobu,Haraldson,F,1987-09-21
997,10100,74365,2000-09-17,2001-09-17,1953-04-21,Hironobu,Haraldson,F,1987-09-21
998,10100,74957,2001-09-17,9999-01-01,1953-04-21,Hironobu,Haraldson,F,1987-09-21


In [27]:
titles = pd.read_csv('C:\\Users\\Kumar\\Desktop\\New folder (2)\\Data-Science\\Python\\EDA\\titles.csv')


In [28]:
titles

,emp_no,title,from_date,to_date
0,10001,Senior Engineer,1986-06-26,9999-01-01
1,10002,Staff,1996-08-03,9999-01-01
2,10003,Senior Engineer,1995-12-03,9999-01-01
3,10004,Engineer,1986-12-01,1995-12-01
4,10004,Senior Engineer,1995-12-01,9999-01-01
...,...,...,...,...
995,10669,Senior Staff,2001-11-17,9999-01-01
996,10669,Staff,1996-11-17,2001-11-17
997,10670,Senior Staff,1993-03-23,9999-01-01
998,10670,Staff,1988-03-23,1993-03-23


In [29]:
titles[titles['emp_no'].isin(emp_no_list)]

,emp_no,title,from_date,to_date
14,10010,Engineer,1996-11-24,9999-01-01
15,10011,Staff,1990-01-22,1996-11-09


In [30]:
# first name , salary (max ) latest salary , hire date , last working if employees present 9999 current emploe , year ex , titels
#left join it better split query


In [36]:
employees_df = pd.read_csv(r'C:\Users\Kumar\Desktop\New folder (2)\Data-Science\Python\EDA\employees.csv')
salaries_df = pd.read_csv(r'C:\Users\Kumar\Desktop\New folder (2)\Data-Science\Python\EDA\Salaries.csv')
titles_df = pd.read_csv(r'C:\Users\Kumar\Desktop\New folder (2)\Data-Science\Python\EDA\titles.csv')

employees_df.to_sql('employees', conn, if_exists='replace', index=False)
salaries_df.to_sql('salaries', conn, if_exists='replace', index=False)
titles_df.to_sql('titles', conn, if_exists='replace', index=False)


1000

In [37]:
df = pd.read_sql_query(query, conn)
print(df)


     first_name      last_name  max_salary  latest_salary   hire_date  \
0         Aamod         Radwan         NaN            NaN  1987-01-27   
1     Abdelaziz        Bisiani         NaN            NaN  1988-09-16   
2    Abdelwaheb    Riesenhuber         NaN            NaN  1990-02-23   
3       Abdulah      Thibadeau         NaN            NaN  1990-12-12   
4       Abdulla  Beutelspacher         NaN            NaN  1992-06-14   
..          ...            ...         ...            ...         ...   
995        Zita       Minakawa         NaN            NaN  1985-09-01   
996     Zongyan       Cusworth         NaN            NaN  1990-04-24   
997      Zorica          Acton         NaN            NaN  1989-08-05   
998       Zsolt        Salinas         NaN            NaN  1985-02-21   
999      Zvonko      Nyanchama     81037.0        81037.0  1989-03-31   

    last_working_date  years_experience            title  
0                None               NaN            Staff  
1    

In [39]:
query = """
SELECT
    e.first_name,
    e.last_name,
    MAX(s.salary) AS max_salary,
    s.salary AS latest_salary,
    e.hire_date,
    CASE
        WHEN s.to_date = '9999-01-01' THEN 'Current Employee'
        ELSE s.to_date
    END AS last_working_date,
    CASE
        WHEN s.to_date = '9999-01-01' THEN strftime('%Y', 'now') - strftime('%Y', e.hire_date)
        ELSE strftime('%Y', s.to_date) - strftime('%Y', e.hire_date)
    END AS years_experience,
    t.title
FROM employees e
LEFT JOIN (
    SELECT emp_no, salary, from_date, to_date
    FROM salaries
    WHERE to_date = '9999-01-01' OR to_date = (
        SELECT MAX(to_date)
        FROM salaries AS s2
        WHERE s2.emp_no = salaries.emp_no
    )
) s ON e.emp_no = s.emp_no
LEFT JOIN (
    SELECT emp_no, title, from_date, to_date
    FROM titles
    WHERE to_date = '9999-01-01' OR to_date = (
        SELECT MAX(to_date)
        FROM titles AS t2
        WHERE t2.emp_no = titles.emp_no
    )
) t ON e.emp_no = t.emp_no
GROUP BY e.emp_no
ORDER BY e.first_name;
"""

# Execute the query
df = pd.read_sql_query(query, conn)
print(df)


     first_name      last_name  max_salary  latest_salary   hire_date  \
0         Aamod         Radwan         NaN            NaN  1987-01-27   
1     Abdelaziz        Bisiani         NaN            NaN  1988-09-16   
2    Abdelwaheb    Riesenhuber         NaN            NaN  1990-02-23   
3       Abdulah      Thibadeau         NaN            NaN  1990-12-12   
4       Abdulla  Beutelspacher         NaN            NaN  1992-06-14   
..          ...            ...         ...            ...         ...   
995        Zita       Minakawa         NaN            NaN  1985-09-01   
996     Zongyan       Cusworth         NaN            NaN  1990-04-24   
997      Zorica          Acton         NaN            NaN  1989-08-05   
998       Zsolt        Salinas         NaN            NaN  1985-02-21   
999      Zvonko      Nyanchama     81037.0        81037.0  1989-03-31   

    last_working_date  years_experience            title  
0                None               NaN            Staff  
1    